In [1]:
# ============================
# VRPTW (Caso base) + Solomon I1
# tw_start/tw_end en HH:MM(:SS) o vacío
# ============================
import pandas as pd
import numpy as np
import math, time, random
from typing import List, Tuple
import folium

# ---------- helpers ----------
def hhmm_to_sec(x):
    """Convierte 'HH:MM' o 'HH:MM:SS' a segundos. Si viene vacío/NaN -> None.
       Si es numérico, lo devuelve como int (asumimos ya en segundos)."""
    if x is None: return None
    if isinstance(x, float) and math.isnan(x): return None
    if isinstance(x, (int, float)): return int(x)
    s = str(x).strip()
    if not s: return None
    parts = s.split(':')
    try:
        if len(parts)==2:
            h, m = map(int, parts); return h*3600 + m*60
        if len(parts)==3:
            h, m, sec = map(int, parts); return h*3600 + m*60 + sec
        # fallback: número en horas
        val = float(s); return int(val*3600)
    except:
        return None

# ---------- carga ----------
dem = pd.read_csv('demands.csv')   # id, latitude, longitude, size, stop_time, tw_start, tw_end
dis = pd.read_csv('distances.csv') # origin_latitude, origin_longitude, destination_latitude, destination_longitude, distance
tim = pd.read_csv('times.csv')     # origin_latitude, origin_longitude, destination_latitude, destination_longitude, time
ov  = pd.read_csv('overview.csv')  # depot_latitude, depot_longitude, start_at, end_at, demands, vehicles
veh = pd.read_csv('vehicles.csv')  # id, capacity
cost= pd.read_csv('costs.csv')     # fixed_route_cost, cost_per_meter, cost_per_vehicle_capacity

# ---------- parámetros globales ----------
depot_lat = float(ov['depot_latitude'].iloc[0])
depot_lon = float(ov['depot_longitude'].iloc[0])
start_at  = hhmm_to_sec(ov['start_at'].iloc[0]) or 0
end_at    = hhmm_to_sec(ov['end_at'].iloc[0])   or (start_at + 8*3600)

# costos
cost_per_meter   = float(cost['cost_per_meter'].iloc[0]) if 'cost_per_meter' in cost.columns else 1.0
fixed_route_cost = float(cost['fixed_route_cost'].iloc[0]) if 'fixed_route_cost' in cost.columns else 0.0
gamma_capacity   = float(cost['cost_per_vehicle_capacity'].iloc[0]) if 'cost_per_vehicle_capacity' in cost.columns else 0.0

# capacidades disponibles (una fila por vehículo en vehicles.csv)
Q_vec = pd.to_numeric(veh['capacity'], errors='coerce').fillna(0).astype(float).tolist()
if len(Q_vec)==0:
    Q_vec = [1e12]  # fallback para no romper (vehículo "ilimitado", no recomendado)

# ---------- índice de nodos: 0=depósito, 1..n=clientes (orden de demands.csv) ----------
n_clients = len(dem)
DEP_OUT, DEP_IN = 0, n_clients+1
n_nodes = n_clients + 2

# coordenadas por nodo
coords = {DEP_OUT: (depot_lat, depot_lon), DEP_IN: (depot_lat, depot_lon)}
for i,(la,lo) in enumerate(zip(dem['latitude'], dem['longitude']), start=1):
    coords[i] = (float(la), float(lo))

# demanda y tiempos de servicio
demand  = np.zeros(n_nodes, dtype=float)
service = np.zeros(n_nodes, dtype=int)
a = np.zeros(n_nodes, dtype=int)                 # ventana inicio
b = np.full(n_nodes, end_at, dtype=int)          # ventana fin

demand[1:n_clients+1]  = pd.to_numeric(dem['size'], errors='coerce').fillna(0).to_numpy()
service[1:n_clients+1] = pd.to_numeric(dem['stop_time'], errors='coerce').fillna(0).astype(int).to_numpy()

# ventanas por cliente (HH:MM:SS o vacío -> usa start_at/end_at globales)
tw_s = dem['tw_start'] if 'tw_start' in dem.columns else pd.Series([None]*n_clients)
tw_e = dem['tw_end']   if 'tw_end'   in dem.columns else pd.Series([None]*n_clients)

for i in range(1, n_clients+1):
    ai = hhmm_to_sec(tw_s.iloc[i-1])
    bi = hhmm_to_sec(tw_e.iloc[i-1])
    a[i] = ai if ai is not None else start_at
    b[i] = bi if bi is not None else end_at

a[DEP_OUT] = start_at
b[DEP_IN]  = end_at
service[DEP_OUT] = 0
service[DEP_IN]  = 0

# ---------- construir matrices desde formato largo por coordenadas ----------
def build_matrix_from_long(df, val_col):
    olat, olon = 'origin_latitude','origin_longitude'
    dlat, dlon = 'destination_latitude','destination_longitude'
    M = np.zeros((n_nodes, n_nodes), dtype=float)
    # pre-hash coords->id
    rev = {coords[i]: i for i in coords}
    miss = 0
    for _,r in df.iterrows():
        try:
            o = (float(r[olat]), float(r[olon]))
            d = (float(r[dlat]), float(r[dlon]))
            i = rev.get(o, None); j = rev.get(d, None)
            if i is None or j is None:
                miss += 1; continue
            M[i,j] = float(r[val_col])
        except Exception:
            miss += 1
    if miss>0:
        print(f"[WARN] {miss} pares origen/destino no matchearon con coords. (Posibles redondeos).")
    return M

dist = build_matrix_from_long(dis, 'distance')
tmat = build_matrix_from_long(tim, 'time')

# ---------- utilidades de horario y costo ----------
def compute_schedule(route: List[int]) -> Tuple[bool, List[int]]:
    """Evalúa factibilidad temporal (TW) de una ruta e informa tiempos de inicio de servicio."""
    T = [0]*len(route)
    T[0] = a[DEP_OUT]
    for k in range(1,len(route)):
        i,j = route[k-1], route[k]
        arrive = T[k-1] + service[i] + int(tmat[i,j])
        T[k] = max(arrive, a[j])   # espera si llega antes
        if T[k] > b[j]:
            return False, []
    return True, T

def route_distance(route: List[int]) -> float:
    return float(np.sum([dist[route[i], route[i+1]] for i in range(len(route)-1)]))

def route_demand(route: List[int]) -> float:
    return float(np.sum([demand[n] for n in route if n not in (DEP_OUT, DEP_IN)]))

def route_operational_cost(route: List[int], veh_capacity: float) -> float:
    """Costo total de una ruta: variable por distancia + fijo por uso + costo por tamaño de vehículo usado."""
    variable = cost_per_meter * route_distance(route)
    fixed    = fixed_route_cost + gamma_capacity * veh_capacity
    return variable + fixed

# ---------- Solomon I1 ----------
ALPHA  = 0.6   # peso del retraso en C1
LAMBDA = 1.0   # peso de lejanía al depósito en C2

def best_insertion_for(route, T_route, u, cap_used, cap_lim):
    best = (False, -1, 1e18, None)
    for p in range(len(route)-1):
        i,j = route[p], route[p+1]
        if cap_used + demand[u] > cap_lim:
            continue
        cand = route[:p+1] + [u] + route[p+1:]
        feas, T_new = compute_schedule(cand)
        if not feas:
            continue
        delta_dist = dist[i,u] + dist[u,j] - dist[i,j]
        T_old_j = T_route[p+1]
        T_new_j = T_new[p+2]
        delta_time = max(0, T_new_j - T_old_j)
        C1 = float(delta_dist + ALPHA*delta_time)
        if C1 < best[2]:
            best = (True, p+1, C1, T_new)
    return best

def solomon_I1():
    d0 = {u: dist[DEP_OUT, u] for u in range(1, n_clients+1)}
    unassigned = set(range(1, n_clients+1))
    routes = []              # lista de tuplas (ruta, tiempos, capacidad_vehículo)
    veh_caps = Q_vec.copy()  # capacidades disponibles
    start = time.time()

    while unassigned:
        if not veh_caps:
            print("[WARN] No quedan vehículos; quedan clientes sin asignar:", len(unassigned))
            break

        cap_lim = veh_caps.pop(0)

        # elegir semilla: más lejos del depósito (regla I1)
        seed = max(unassigned, key=lambda u: d0[u])
        r = [DEP_OUT, seed, DEP_IN]
        feas, T_r = compute_schedule(r)
        if (not feas) or (demand[seed] > cap_lim):
            feasible_seeds = [u for u in unassigned
                              if demand[u] <= cap_lim and compute_schedule([DEP_OUT,u,DEP_IN])[0]]
            if feasible_seeds:
                seed = max(feasible_seeds, key=lambda u: d0[u])
                r = [DEP_OUT, seed, DEP_IN]
                feas, T_r = compute_schedule(r)
            else:
                print(f"[INFO] Ningún cliente cabe en vehículo cap={cap_lim}. Se descarta ese vehículo.")
                continue

        cap_used = demand[seed]
        unassigned.remove(seed)

        # inserciones secuenciales
        while True:
            cand = []
            for u in list(unassigned):
                ok,pos,C1u,Tn = best_insertion_for(r, T_r, u, cap_used, cap_lim)
                if ok:
                    cand.append((u,pos,C1u,Tn))
            if not cand:
                break
            # C2: balancea lejanía al depósito vs. incremento de C1
            u,pos,C1u,Tn = max(cand, key=lambda x: LAMBDA*d0[x[0]] - x[2])
            r = r[:pos] + [u] + r[pos:]
            T_r = Tn
            cap_used += demand[u]
            unassigned.remove(u)

        routes.append((r, T_r, cap_lim))

    elapsed = time.time()-start
    return routes, elapsed

routes, elapsed = solomon_I1()

# ---------- reporte ----------
total_dist = sum(route_distance(r) for r,_,_ in routes)
# costo operacional: suma de costo variable por distancia + (F + gamma*Q_k) por cada vehículo usado
total_cost = sum(route_operational_cost(r, cap) for r,_,cap in routes)

print("===== SOLUCIÓN SOLOMON I1 (caso base con costos completos) =====")
print(f"Vehículos usados: {len(routes)}")
print(f"Distancia total : {total_dist:,.0f} m")
print(f"Costo total     : {total_cost:,.2f}")
print(f"Tiempo ejec     : {elapsed:.2f} s\n")

for k,(r,T,cap_lim) in enumerate(routes, start=1):
    print(f"Vehículo {k} (cap={cap_lim:.0f}) | Dist: {route_distance(r):,.0f} m | Demanda: {route_demand(r):,.0f}")
    print("Ruta:", " -> ".join(map(str,r)))
    # print("Tiempos:", T)
    print()

# ---------- mapa ----------
m = folium.Map(location=[depot_lat, depot_lon], zoom_start=11)
folium.Marker([depot_lat, depot_lon], tooltip="Depósito", icon=folium.Icon(color='blue')).add_to(m)
palette = ["#"+''.join(random.choice('0123456789ABCDEF') for _ in range(6)) for _ in range(len(routes))]

for idx,(r,_,_) in enumerate(routes):
    pts = [coords[n] for n in r]
    folium.PolyLine(pts, color=palette[idx], weight=3, opacity=0.9).add_to(m)
m.save("routes_map.html")
print("Mapa guardado como routes_map.html")


===== SOLUCIÓN SOLOMON I1 (caso base con costos completos) =====
Vehículos usados: 3
Distancia total : 182,583 m
Costo total     : 309,244.88
Tiempo ejec     : 0.16 s

Vehículo 1 (cap=3800) | Dist: 51,681 m | Demanda: 3,720
Ruta: 0 -> 1 -> 3 -> 38 -> 31 -> 32 -> 41 -> 39 -> 35 -> 11 -> 17 -> 13 -> 48 -> 42 -> 36 -> 28 -> 29 -> 5 -> 37 -> 4 -> 47 -> 2 -> 49

Vehículo 2 (cap=3900) | Dist: 52,071 m | Demanda: 3,840
Ruta: 0 -> 8 -> 9 -> 10 -> 7 -> 27 -> 34 -> 21 -> 43 -> 22 -> 19 -> 15 -> 30 -> 23 -> 6 -> 33 -> 16 -> 12 -> 20 -> 46 -> 18 -> 49

Vehículo 3 (cap=4000) | Dist: 78,832 m | Demanda: 3,240
Ruta: 0 -> 25 -> 45 -> 40 -> 24 -> 44 -> 26 -> 14 -> 49

Mapa guardado como routes_map.html


#INSTANCIA 2

In [2]:
# ============================
# VRPTW (Caso base) + Solomon I1
# tw_start/tw_end en HH:MM(:SS) o vacío
# ============================
import pandas as pd
import numpy as np
import math, time, random
from typing import List, Tuple
import folium

# ---------- helpers ----------
def hhmm_to_sec(x):
    """Convierte 'HH:MM' o 'HH:MM:SS' a segundos. Si viene vacío/NaN -> None.
       Si es numérico, lo devuelve como int (asumimos ya en segundos)."""
    if x is None: return None
    if isinstance(x, float) and math.isnan(x): return None
    if isinstance(x, (int, float)): return int(x)
    s = str(x).strip()
    if not s: return None
    parts = s.split(':')
    try:
        if len(parts)==2:
            h, m = map(int, parts); return h*3600 + m*60
        if len(parts)==3:
            h, m, sec = map(int, parts); return h*3600 + m*60 + sec
        # fallback: número en horas
        val = float(s); return int(val*3600)
    except:
        return None

# ---------- carga ----------
dem = pd.read_csv('demands.csv')   # id, latitude, longitude, size, stop_time, tw_start, tw_end
dis = pd.read_csv('distances.csv') # origin_latitude, origin_longitude, destination_latitude, destination_longitude, distance
tim = pd.read_csv('times.csv')     # origin_latitude, origin_longitude, destination_latitude, destination_longitude, time
ov  = pd.read_csv('overview.csv')  # depot_latitude, depot_longitude, start_at, end_at, demands, vehicles
veh = pd.read_csv('vehicles.csv')  # id, capacity
cost= pd.read_csv('costs.csv')     # fixed_route_cost, cost_per_meter, cost_per_vehicle_capacity

# ---------- parámetros globales ----------
depot_lat = float(ov['depot_latitude'].iloc[0])
depot_lon = float(ov['depot_longitude'].iloc[0])
start_at  = hhmm_to_sec(ov['start_at'].iloc[0]) or 0
end_at    = hhmm_to_sec(ov['end_at'].iloc[0])   or (start_at + 8*3600)

# costos
cost_per_meter   = float(cost['cost_per_meter'].iloc[0]) if 'cost_per_meter' in cost.columns else 1.0
fixed_route_cost = float(cost['fixed_route_cost'].iloc[0]) if 'fixed_route_cost' in cost.columns else 0.0
gamma_capacity   = float(cost['cost_per_vehicle_capacity'].iloc[0]) if 'cost_per_vehicle_capacity' in cost.columns else 0.0

# capacidades disponibles (una fila por vehículo en vehicles.csv)
Q_vec = pd.to_numeric(veh['capacity'], errors='coerce').fillna(0).astype(float).tolist()
if len(Q_vec)==0:
    Q_vec = [1e12]  # fallback para no romper (vehículo "ilimitado", no recomendado)

# ---------- índice de nodos: 0=depósito, 1..n=clientes (orden de demands.csv) ----------
n_clients = len(dem)
DEP_OUT, DEP_IN = 0, n_clients+1
n_nodes = n_clients + 2

# coordenadas por nodo
coords = {DEP_OUT: (depot_lat, depot_lon), DEP_IN: (depot_lat, depot_lon)}
for i,(la,lo) in enumerate(zip(dem['latitude'], dem['longitude']), start=1):
    coords[i] = (float(la), float(lo))

# demanda y tiempos de servicio
demand  = np.zeros(n_nodes, dtype=float)
service = np.zeros(n_nodes, dtype=int)
a = np.zeros(n_nodes, dtype=int)                 # ventana inicio
b = np.full(n_nodes, end_at, dtype=int)          # ventana fin

demand[1:n_clients+1]  = pd.to_numeric(dem['size'], errors='coerce').fillna(0).to_numpy()
service[1:n_clients+1] = pd.to_numeric(dem['stop_time'], errors='coerce').fillna(0).astype(int).to_numpy()

# ventanas por cliente (HH:MM:SS o vacío -> usa start_at/end_at globales)
tw_s = dem['tw_start'] if 'tw_start' in dem.columns else pd.Series([None]*n_clients)
tw_e = dem['tw_end']   if 'tw_end'   in dem.columns else pd.Series([None]*n_clients)

for i in range(1, n_clients+1):
    ai = hhmm_to_sec(tw_s.iloc[i-1])
    bi = hhmm_to_sec(tw_e.iloc[i-1])
    a[i] = ai if ai is not None else start_at
    b[i] = bi if bi is not None else end_at

a[DEP_OUT] = start_at
b[DEP_IN]  = end_at
service[DEP_OUT] = 0
service[DEP_IN]  = 0

# ---------- construir matrices desde formato largo por coordenadas ----------
def build_matrix_from_long(df, val_col):
    olat, olon = 'origin_latitude','origin_longitude'
    dlat, dlon = 'destination_latitude','destination_longitude'
    M = np.zeros((n_nodes, n_nodes), dtype=float)
    # pre-hash coords->id
    rev = {coords[i]: i for i in coords}
    miss = 0
    for _,r in df.iterrows():
        try:
            o = (float(r[olat]), float(r[olon]))
            d = (float(r[dlat]), float(r[dlon]))
            i = rev.get(o, None); j = rev.get(d, None)
            if i is None or j is None:
                miss += 1; continue
            M[i,j] = float(r[val_col])
        except Exception:
            miss += 1
    if miss>0:
        print(f"[WARN] {miss} pares origen/destino no matchearon con coords. (Posibles redondeos).")
    return M

dist = build_matrix_from_long(dis, 'distance')
tmat = build_matrix_from_long(tim, 'time')

# ---------- utilidades de horario y costo ----------
def compute_schedule(route: List[int]) -> Tuple[bool, List[int]]:
    """Evalúa factibilidad temporal (TW) de una ruta e informa tiempos de inicio de servicio."""
    T = [0]*len(route)
    T[0] = a[DEP_OUT]
    for k in range(1,len(route)):
        i,j = route[k-1], route[k]
        arrive = T[k-1] + service[i] + int(tmat[i,j])
        T[k] = max(arrive, a[j])   # espera si llega antes
        if T[k] > b[j]:
            return False, []
    return True, T

def route_distance(route: List[int]) -> float:
    return float(np.sum([dist[route[i], route[i+1]] for i in range(len(route)-1)]))

def route_demand(route: List[int]) -> float:
    return float(np.sum([demand[n] for n in route if n not in (DEP_OUT, DEP_IN)]))

def route_operational_cost(route: List[int], veh_capacity: float) -> float:
    """Costo total de una ruta: variable por distancia + fijo por uso + costo por tamaño de vehículo usado."""
    variable = cost_per_meter * route_distance(route)
    fixed    = fixed_route_cost + gamma_capacity * veh_capacity
    return variable + fixed

# ---------- Solomon I1 ----------
ALPHA  = 0.6   # peso del retraso en C1
LAMBDA = 1.0   # peso de lejanía al depósito en C2

def best_insertion_for(route, T_route, u, cap_used, cap_lim):
    best = (False, -1, 1e18, None)
    for p in range(len(route)-1):
        i,j = route[p], route[p+1]
        if cap_used + demand[u] > cap_lim:
            continue
        cand = route[:p+1] + [u] + route[p+1:]
        feas, T_new = compute_schedule(cand)
        if not feas:
            continue
        delta_dist = dist[i,u] + dist[u,j] - dist[i,j]
        T_old_j = T_route[p+1]
        T_new_j = T_new[p+2]
        delta_time = max(0, T_new_j - T_old_j)
        C1 = float(delta_dist + ALPHA*delta_time)
        if C1 < best[2]:
            best = (True, p+1, C1, T_new)
    return best

def solomon_I1():
    d0 = {u: dist[DEP_OUT, u] for u in range(1, n_clients+1)}
    unassigned = set(range(1, n_clients+1))
    routes = []              # lista de tuplas (ruta, tiempos, capacidad_vehículo)
    veh_caps = Q_vec.copy()  # capacidades disponibles
    start = time.time()

    while unassigned:
        if not veh_caps:
            print("[WARN] No quedan vehículos; quedan clientes sin asignar:", len(unassigned))
            break

        cap_lim = veh_caps.pop(0)

        # elegir semilla: más lejos del depósito (regla I1)
        seed = max(unassigned, key=lambda u: d0[u])
        r = [DEP_OUT, seed, DEP_IN]
        feas, T_r = compute_schedule(r)
        if (not feas) or (demand[seed] > cap_lim):
            feasible_seeds = [u for u in unassigned
                              if demand[u] <= cap_lim and compute_schedule([DEP_OUT,u,DEP_IN])[0]]
            if feasible_seeds:
                seed = max(feasible_seeds, key=lambda u: d0[u])
                r = [DEP_OUT, seed, DEP_IN]
                feas, T_r = compute_schedule(r)
            else:
                print(f"[INFO] Ningún cliente cabe en vehículo cap={cap_lim}. Se descarta ese vehículo.")
                continue

        cap_used = demand[seed]
        unassigned.remove(seed)

        # inserciones secuenciales
        while True:
            cand = []
            for u in list(unassigned):
                ok,pos,C1u,Tn = best_insertion_for(r, T_r, u, cap_used, cap_lim)
                if ok:
                    cand.append((u,pos,C1u,Tn))
            if not cand:
                break
            # C2: balancea lejanía al depósito vs. incremento de C1
            u,pos,C1u,Tn = max(cand, key=lambda x: LAMBDA*d0[x[0]] - x[2])
            r = r[:pos] + [u] + r[pos:]
            T_r = Tn
            cap_used += demand[u]
            unassigned.remove(u)

        routes.append((r, T_r, cap_lim))

    elapsed = time.time()-start
    return routes, elapsed

routes, elapsed = solomon_I1()

# ---------- reporte ----------
total_dist = sum(route_distance(r) for r,_,_ in routes)
# costo operacional: suma de costo variable por distancia + (F + gamma*Q_k) por cada vehículo usado
total_cost = sum(route_operational_cost(r, cap) for r,_,cap in routes)

print("===== SOLUCIÓN SOLOMON I1 (caso base con costos completos) =====")
print(f"Vehículos usados: {len(routes)}")
print(f"Distancia total : {total_dist:,.0f} m")
print(f"Costo total     : {total_cost:,.2f}")
print(f"Tiempo ejec     : {elapsed:.2f} s\n")

for k,(r,T,cap_lim) in enumerate(routes, start=1):
    print(f"Vehículo {k} (cap={cap_lim:.0f}) | Dist: {route_distance(r):,.0f} m | Demanda: {route_demand(r):,.0f}")
    print("Ruta:", " -> ".join(map(str,r)))
    # print("Tiempos:", T)
    print()

# ---------- mapa ----------
m = folium.Map(location=[depot_lat, depot_lon], zoom_start=11)
folium.Marker([depot_lat, depot_lon], tooltip="Depósito", icon=folium.Icon(color='blue')).add_to(m)
palette = ["#"+''.join(random.choice('0123456789ABCDEF') for _ in range(6)) for _ in range(len(routes))]

for idx,(r,_,_) in enumerate(routes):
    pts = [coords[n] for n in r]
    folium.PolyLine(pts, color=palette[idx], weight=3, opacity=0.9).add_to(m)
m.save("routes_map.html")
print("Mapa guardado como routes_map.html")


[INFO] Ningún cliente cabe en vehículo cap=4500.0. Se descarta ese vehículo.
[INFO] Ningún cliente cabe en vehículo cap=4500.0. Se descarta ese vehículo.
[INFO] Ningún cliente cabe en vehículo cap=4500.0. Se descarta ese vehículo.
[INFO] Ningún cliente cabe en vehículo cap=4508.0. Se descarta ese vehículo.
[INFO] Ningún cliente cabe en vehículo cap=4646.0. Se descarta ese vehículo.
[INFO] Ningún cliente cabe en vehículo cap=4800.0. Se descarta ese vehículo.
[INFO] Ningún cliente cabe en vehículo cap=5000.0. Se descarta ese vehículo.
[INFO] Ningún cliente cabe en vehículo cap=5000.0. Se descarta ese vehículo.
[INFO] Ningún cliente cabe en vehículo cap=5079.0. Se descarta ese vehículo.
[INFO] Ningún cliente cabe en vehículo cap=5200.0. Se descarta ese vehículo.
[INFO] Ningún cliente cabe en vehículo cap=5200.0. Se descarta ese vehículo.
[INFO] Ningún cliente cabe en vehículo cap=5200.0. Se descarta ese vehículo.
[INFO] Ningún cliente cabe en vehículo cap=5200.0. Se descarta ese vehículo.

#INSTANCIA 3

In [3]:
# ============================
# VRPTW (Caso base) + Solomon I1
# tw_start/tw_end en HH:MM(:SS) o vacío
# ============================
import pandas as pd
import numpy as np
import math, time, random
from typing import List, Tuple
import folium

# ---------- helpers ----------
def hhmm_to_sec(x):
    """Convierte 'HH:MM' o 'HH:MM:SS' a segundos. Si viene vacío/NaN -> None.
       Si es numérico, lo devuelve como int (asumimos ya en segundos)."""
    if x is None: return None
    if isinstance(x, float) and math.isnan(x): return None
    if isinstance(x, (int, float)): return int(x)
    s = str(x).strip()
    if not s: return None
    parts = s.split(':')
    try:
        if len(parts)==2:
            h, m = map(int, parts); return h*3600 + m*60
        if len(parts)==3:
            h, m, sec = map(int, parts); return h*3600 + m*60 + sec
        # fallback: número en horas
        val = float(s); return int(val*3600)
    except:
        return None

# ---------- carga ----------
dem = pd.read_csv('demands.csv')   # id, latitude, longitude, size, stop_time, tw_start, tw_end
dis = pd.read_csv('distances.csv') # origin_latitude, origin_longitude, destination_latitude, destination_longitude, distance
tim = pd.read_csv('times.csv')     # origin_latitude, origin_longitude, destination_latitude, destination_longitude, time
ov  = pd.read_csv('overview.csv')  # depot_latitude, depot_longitude, start_at, end_at, demands, vehicles
veh = pd.read_csv('vehicles.csv')  # id, capacity
cost= pd.read_csv('costs.csv')     # fixed_route_cost, cost_per_meter, cost_per_vehicle_capacity

# ---------- parámetros globales ----------
depot_lat = float(ov['depot_latitude'].iloc[0])
depot_lon = float(ov['depot_longitude'].iloc[0])
start_at  = hhmm_to_sec(ov['start_at'].iloc[0]) or 0
end_at    = hhmm_to_sec(ov['end_at'].iloc[0])   or (start_at + 8*3600)

# costos
cost_per_meter   = float(cost['cost_per_meter'].iloc[0]) if 'cost_per_meter' in cost.columns else 1.0
fixed_route_cost = float(cost['fixed_route_cost'].iloc[0]) if 'fixed_route_cost' in cost.columns else 0.0
gamma_capacity   = float(cost['cost_per_vehicle_capacity'].iloc[0]) if 'cost_per_vehicle_capacity' in cost.columns else 0.0

# capacidades disponibles (una fila por vehículo en vehicles.csv)
Q_vec = pd.to_numeric(veh['capacity'], errors='coerce').fillna(0).astype(float).tolist()
if len(Q_vec)==0:
    Q_vec = [1e12]  # fallback para no romper (vehículo "ilimitado", no recomendado)

# ---------- índice de nodos: 0=depósito, 1..n=clientes (orden de demands.csv) ----------
n_clients = len(dem)
DEP_OUT, DEP_IN = 0, n_clients+1
n_nodes = n_clients + 2

# coordenadas por nodo
coords = {DEP_OUT: (depot_lat, depot_lon), DEP_IN: (depot_lat, depot_lon)}
for i,(la,lo) in enumerate(zip(dem['latitude'], dem['longitude']), start=1):
    coords[i] = (float(la), float(lo))

# demanda y tiempos de servicio
demand  = np.zeros(n_nodes, dtype=float)
service = np.zeros(n_nodes, dtype=int)
a = np.zeros(n_nodes, dtype=int)                 # ventana inicio
b = np.full(n_nodes, end_at, dtype=int)          # ventana fin

demand[1:n_clients+1]  = pd.to_numeric(dem['size'], errors='coerce').fillna(0).to_numpy()
service[1:n_clients+1] = pd.to_numeric(dem['stop_time'], errors='coerce').fillna(0).astype(int).to_numpy()

# ventanas por cliente (HH:MM:SS o vacío -> usa start_at/end_at globales)
tw_s = dem['tw_start'] if 'tw_start' in dem.columns else pd.Series([None]*n_clients)
tw_e = dem['tw_end']   if 'tw_end'   in dem.columns else pd.Series([None]*n_clients)

for i in range(1, n_clients+1):
    ai = hhmm_to_sec(tw_s.iloc[i-1])
    bi = hhmm_to_sec(tw_e.iloc[i-1])
    a[i] = ai if ai is not None else start_at
    b[i] = bi if bi is not None else end_at

a[DEP_OUT] = start_at
b[DEP_IN]  = end_at
service[DEP_OUT] = 0
service[DEP_IN]  = 0

# ---------- construir matrices desde formato largo por coordenadas ----------
def build_matrix_from_long(df, val_col):
    olat, olon = 'origin_latitude','origin_longitude'
    dlat, dlon = 'destination_latitude','destination_longitude'
    M = np.zeros((n_nodes, n_nodes), dtype=float)
    # pre-hash coords->id
    rev = {coords[i]: i for i in coords}
    miss = 0
    for _,r in df.iterrows():
        try:
            o = (float(r[olat]), float(r[olon]))
            d = (float(r[dlat]), float(r[dlon]))
            i = rev.get(o, None); j = rev.get(d, None)
            if i is None or j is None:
                miss += 1; continue
            M[i,j] = float(r[val_col])
        except Exception:
            miss += 1
    if miss>0:
        print(f"[WARN] {miss} pares origen/destino no matchearon con coords. (Posibles redondeos).")
    return M

dist = build_matrix_from_long(dis, 'distance')
tmat = build_matrix_from_long(tim, 'time')

# ---------- utilidades de horario y costo ----------
def compute_schedule(route: List[int]) -> Tuple[bool, List[int]]:
    """Evalúa factibilidad temporal (TW) de una ruta e informa tiempos de inicio de servicio."""
    T = [0]*len(route)
    T[0] = a[DEP_OUT]
    for k in range(1,len(route)):
        i,j = route[k-1], route[k]
        arrive = T[k-1] + service[i] + int(tmat[i,j])
        T[k] = max(arrive, a[j])   # espera si llega antes
        if T[k] > b[j]:
            return False, []
    return True, T

def route_distance(route: List[int]) -> float:
    return float(np.sum([dist[route[i], route[i+1]] for i in range(len(route)-1)]))

def route_demand(route: List[int]) -> float:
    return float(np.sum([demand[n] for n in route if n not in (DEP_OUT, DEP_IN)]))

def route_operational_cost(route: List[int], veh_capacity: float) -> float:
    """Costo total de una ruta: variable por distancia + fijo por uso + costo por tamaño de vehículo usado."""
    variable = cost_per_meter * route_distance(route)
    fixed    = fixed_route_cost + gamma_capacity * veh_capacity
    return variable + fixed

# ---------- Solomon I1 ----------
ALPHA  = 0.6   # peso del retraso en C1
LAMBDA = 1.0   # peso de lejanía al depósito en C2

def best_insertion_for(route, T_route, u, cap_used, cap_lim):
    best = (False, -1, 1e18, None)
    for p in range(len(route)-1):
        i,j = route[p], route[p+1]
        if cap_used + demand[u] > cap_lim:
            continue
        cand = route[:p+1] + [u] + route[p+1:]
        feas, T_new = compute_schedule(cand)
        if not feas:
            continue
        delta_dist = dist[i,u] + dist[u,j] - dist[i,j]
        T_old_j = T_route[p+1]
        T_new_j = T_new[p+2]
        delta_time = max(0, T_new_j - T_old_j)
        C1 = float(delta_dist + ALPHA*delta_time)
        if C1 < best[2]:
            best = (True, p+1, C1, T_new)
    return best

def solomon_I1():
    d0 = {u: dist[DEP_OUT, u] for u in range(1, n_clients+1)}
    unassigned = set(range(1, n_clients+1))
    routes = []              # lista de tuplas (ruta, tiempos, capacidad_vehículo)
    veh_caps = Q_vec.copy()  # capacidades disponibles
    start = time.time()

    while unassigned:
        if not veh_caps:
            print("[WARN] No quedan vehículos; quedan clientes sin asignar:", len(unassigned))
            break

        cap_lim = veh_caps.pop(0)

        # elegir semilla: más lejos del depósito (regla I1)
        seed = max(unassigned, key=lambda u: d0[u])
        r = [DEP_OUT, seed, DEP_IN]
        feas, T_r = compute_schedule(r)
        if (not feas) or (demand[seed] > cap_lim):
            feasible_seeds = [u for u in unassigned
                              if demand[u] <= cap_lim and compute_schedule([DEP_OUT,u,DEP_IN])[0]]
            if feasible_seeds:
                seed = max(feasible_seeds, key=lambda u: d0[u])
                r = [DEP_OUT, seed, DEP_IN]
                feas, T_r = compute_schedule(r)
            else:
                print(f"[INFO] Ningún cliente cabe en vehículo cap={cap_lim}. Se descarta ese vehículo.")
                continue

        cap_used = demand[seed]
        unassigned.remove(seed)

        # inserciones secuenciales
        while True:
            cand = []
            for u in list(unassigned):
                ok,pos,C1u,Tn = best_insertion_for(r, T_r, u, cap_used, cap_lim)
                if ok:
                    cand.append((u,pos,C1u,Tn))
            if not cand:
                break
            # C2: balancea lejanía al depósito vs. incremento de C1
            u,pos,C1u,Tn = max(cand, key=lambda x: LAMBDA*d0[x[0]] - x[2])
            r = r[:pos] + [u] + r[pos:]
            T_r = Tn
            cap_used += demand[u]
            unassigned.remove(u)

        routes.append((r, T_r, cap_lim))

    elapsed = time.time()-start
    return routes, elapsed

routes, elapsed = solomon_I1()

# ---------- reporte ----------
total_dist = sum(route_distance(r) for r,_,_ in routes)
# costo operacional: suma de costo variable por distancia + (F + gamma*Q_k) por cada vehículo usado
total_cost = sum(route_operational_cost(r, cap) for r,_,cap in routes)

print("===== SOLUCIÓN SOLOMON I1 (caso base con costos completos) =====")
print(f"Vehículos usados: {len(routes)}")
print(f"Distancia total : {total_dist:,.0f} m")
print(f"Costo total     : {total_cost:,.2f}")
print(f"Tiempo ejec     : {elapsed:.2f} s\n")

for k,(r,T,cap_lim) in enumerate(routes, start=1):
    print(f"Vehículo {k} (cap={cap_lim:.0f}) | Dist: {route_distance(r):,.0f} m | Demanda: {route_demand(r):,.0f}")
    print("Ruta:", " -> ".join(map(str,r)))
    # print("Tiempos:", T)
    print()

# ---------- mapa ----------
m = folium.Map(location=[depot_lat, depot_lon], zoom_start=11)
folium.Marker([depot_lat, depot_lon], tooltip="Depósito", icon=folium.Icon(color='blue')).add_to(m)
palette = ["#"+''.join(random.choice('0123456789ABCDEF') for _ in range(6)) for _ in range(len(routes))]

for idx,(r,_,_) in enumerate(routes):
    pts = [coords[n] for n in r]
    folium.PolyLine(pts, color=palette[idx], weight=3, opacity=0.9).add_to(m)
m.save("routes_map.html")
print("Mapa guardado como routes_map.html")


[INFO] Ningún cliente cabe en vehículo cap=5200.0. Se descarta ese vehículo.
[INFO] Ningún cliente cabe en vehículo cap=5200.0. Se descarta ese vehículo.
[INFO] Ningún cliente cabe en vehículo cap=5200.0. Se descarta ese vehículo.
[INFO] Ningún cliente cabe en vehículo cap=5200.0. Se descarta ese vehículo.
[INFO] Ningún cliente cabe en vehículo cap=5200.0. Se descarta ese vehículo.
[INFO] Ningún cliente cabe en vehículo cap=5224.0. Se descarta ese vehículo.
[INFO] Ningún cliente cabe en vehículo cap=5300.0. Se descarta ese vehículo.
[INFO] Ningún cliente cabe en vehículo cap=5300.0. Se descarta ese vehículo.
[INFO] Ningún cliente cabe en vehículo cap=5472.0. Se descarta ese vehículo.
[INFO] Ningún cliente cabe en vehículo cap=5500.0. Se descarta ese vehículo.
[INFO] Ningún cliente cabe en vehículo cap=6650.0. Se descarta ese vehículo.
[INFO] Ningún cliente cabe en vehículo cap=6820.0. Se descarta ese vehículo.
[INFO] Ningún cliente cabe en vehículo cap=7000.0. Se descarta ese vehículo.

#INSTANCIA 4

In [4]:
# ============================
# VRPTW (Caso base) + Solomon I1
# tw_start/tw_end en HH:MM(:SS) o vacío
# ============================
import pandas as pd
import numpy as np
import math, time, random
from typing import List, Tuple
import folium

# ---------- helpers ----------
def hhmm_to_sec(x):
    """Convierte 'HH:MM' o 'HH:MM:SS' a segundos. Si viene vacío/NaN -> None.
       Si es numérico, lo devuelve como int (asumimos ya en segundos)."""
    if x is None: return None
    if isinstance(x, float) and math.isnan(x): return None
    if isinstance(x, (int, float)): return int(x)
    s = str(x).strip()
    if not s: return None
    parts = s.split(':')
    try:
        if len(parts)==2:
            h, m = map(int, parts); return h*3600 + m*60
        if len(parts)==3:
            h, m, sec = map(int, parts); return h*3600 + m*60 + sec
        # fallback: número en horas
        val = float(s); return int(val*3600)
    except:
        return None

# ---------- carga ----------
dem = pd.read_csv('demands.csv')   # id, latitude, longitude, size, stop_time, tw_start, tw_end
dis = pd.read_csv('distances.csv') # origin_latitude, origin_longitude, destination_latitude, destination_longitude, distance
tim = pd.read_csv('times.csv')     # origin_latitude, origin_longitude, destination_latitude, destination_longitude, time
ov  = pd.read_csv('overview.csv')  # depot_latitude, depot_longitude, start_at, end_at, demands, vehicles
veh = pd.read_csv('vehicles.csv')  # id, capacity
cost= pd.read_csv('costs.csv')     # fixed_route_cost, cost_per_meter, cost_per_vehicle_capacity

# ---------- parámetros globales ----------
depot_lat = float(ov['depot_latitude'].iloc[0])
depot_lon = float(ov['depot_longitude'].iloc[0])
start_at  = hhmm_to_sec(ov['start_at'].iloc[0]) or 0
end_at    = hhmm_to_sec(ov['end_at'].iloc[0])   or (start_at + 8*3600)

# costos
cost_per_meter   = float(cost['cost_per_meter'].iloc[0]) if 'cost_per_meter' in cost.columns else 1.0
fixed_route_cost = float(cost['fixed_route_cost'].iloc[0]) if 'fixed_route_cost' in cost.columns else 0.0
gamma_capacity   = float(cost['cost_per_vehicle_capacity'].iloc[0]) if 'cost_per_vehicle_capacity' in cost.columns else 0.0

# capacidades disponibles (una fila por vehículo en vehicles.csv)
Q_vec = pd.to_numeric(veh['capacity'], errors='coerce').fillna(0).astype(float).tolist()
if len(Q_vec)==0:
    Q_vec = [1e12]  # fallback para no romper (vehículo "ilimitado", no recomendado)

# ---------- índice de nodos: 0=depósito, 1..n=clientes (orden de demands.csv) ----------
n_clients = len(dem)
DEP_OUT, DEP_IN = 0, n_clients+1
n_nodes = n_clients + 2

# coordenadas por nodo
coords = {DEP_OUT: (depot_lat, depot_lon), DEP_IN: (depot_lat, depot_lon)}
for i,(la,lo) in enumerate(zip(dem['latitude'], dem['longitude']), start=1):
    coords[i] = (float(la), float(lo))

# demanda y tiempos de servicio
demand  = np.zeros(n_nodes, dtype=float)
service = np.zeros(n_nodes, dtype=int)
a = np.zeros(n_nodes, dtype=int)                 # ventana inicio
b = np.full(n_nodes, end_at, dtype=int)          # ventana fin

demand[1:n_clients+1]  = pd.to_numeric(dem['size'], errors='coerce').fillna(0).to_numpy()
service[1:n_clients+1] = pd.to_numeric(dem['stop_time'], errors='coerce').fillna(0).astype(int).to_numpy()

# ventanas por cliente (HH:MM:SS o vacío -> usa start_at/end_at globales)
tw_s = dem['tw_start'] if 'tw_start' in dem.columns else pd.Series([None]*n_clients)
tw_e = dem['tw_end']   if 'tw_end'   in dem.columns else pd.Series([None]*n_clients)

for i in range(1, n_clients+1):
    ai = hhmm_to_sec(tw_s.iloc[i-1])
    bi = hhmm_to_sec(tw_e.iloc[i-1])
    a[i] = ai if ai is not None else start_at
    b[i] = bi if bi is not None else end_at

a[DEP_OUT] = start_at
b[DEP_IN]  = end_at
service[DEP_OUT] = 0
service[DEP_IN]  = 0

# ---------- construir matrices desde formato largo por coordenadas ----------
def build_matrix_from_long(df, val_col):
    olat, olon = 'origin_latitude','origin_longitude'
    dlat, dlon = 'destination_latitude','destination_longitude'
    M = np.zeros((n_nodes, n_nodes), dtype=float)
    # pre-hash coords->id
    rev = {coords[i]: i for i in coords}
    miss = 0
    for _,r in df.iterrows():
        try:
            o = (float(r[olat]), float(r[olon]))
            d = (float(r[dlat]), float(r[dlon]))
            i = rev.get(o, None); j = rev.get(d, None)
            if i is None or j is None:
                miss += 1; continue
            M[i,j] = float(r[val_col])
        except Exception:
            miss += 1
    if miss>0:
        print(f"[WARN] {miss} pares origen/destino no matchearon con coords. (Posibles redondeos).")
    return M

dist = build_matrix_from_long(dis, 'distance')
tmat = build_matrix_from_long(tim, 'time')

# ---------- utilidades de horario y costo ----------
def compute_schedule(route: List[int]) -> Tuple[bool, List[int]]:
    """Evalúa factibilidad temporal (TW) de una ruta e informa tiempos de inicio de servicio."""
    T = [0]*len(route)
    T[0] = a[DEP_OUT]
    for k in range(1,len(route)):
        i,j = route[k-1], route[k]
        arrive = T[k-1] + service[i] + int(tmat[i,j])
        T[k] = max(arrive, a[j])   # espera si llega antes
        if T[k] > b[j]:
            return False, []
    return True, T

def route_distance(route: List[int]) -> float:
    return float(np.sum([dist[route[i], route[i+1]] for i in range(len(route)-1)]))

def route_demand(route: List[int]) -> float:
    return float(np.sum([demand[n] for n in route if n not in (DEP_OUT, DEP_IN)]))

def route_operational_cost(route: List[int], veh_capacity: float) -> float:
    """Costo total de una ruta: variable por distancia + fijo por uso + costo por tamaño de vehículo usado."""
    variable = cost_per_meter * route_distance(route)
    fixed    = fixed_route_cost + gamma_capacity * veh_capacity
    return variable + fixed

# ---------- Solomon I1 ----------
ALPHA  = 0.6   # peso del retraso en C1
LAMBDA = 1.0   # peso de lejanía al depósito en C2

def best_insertion_for(route, T_route, u, cap_used, cap_lim):
    best = (False, -1, 1e18, None)
    for p in range(len(route)-1):
        i,j = route[p], route[p+1]
        if cap_used + demand[u] > cap_lim:
            continue
        cand = route[:p+1] + [u] + route[p+1:]
        feas, T_new = compute_schedule(cand)
        if not feas:
            continue
        delta_dist = dist[i,u] + dist[u,j] - dist[i,j]
        T_old_j = T_route[p+1]
        T_new_j = T_new[p+2]
        delta_time = max(0, T_new_j - T_old_j)
        C1 = float(delta_dist + ALPHA*delta_time)
        if C1 < best[2]:
            best = (True, p+1, C1, T_new)
    return best

def solomon_I1():
    d0 = {u: dist[DEP_OUT, u] for u in range(1, n_clients+1)}
    unassigned = set(range(1, n_clients+1))
    routes = []              # lista de tuplas (ruta, tiempos, capacidad_vehículo)
    veh_caps = Q_vec.copy()  # capacidades disponibles
    start = time.time()

    while unassigned:
        if not veh_caps:
            print("[WARN] No quedan vehículos; quedan clientes sin asignar:", len(unassigned))
            break

        cap_lim = veh_caps.pop(0)

        # elegir semilla: más lejos del depósito (regla I1)
        seed = max(unassigned, key=lambda u: d0[u])
        r = [DEP_OUT, seed, DEP_IN]
        feas, T_r = compute_schedule(r)
        if (not feas) or (demand[seed] > cap_lim):
            feasible_seeds = [u for u in unassigned
                              if demand[u] <= cap_lim and compute_schedule([DEP_OUT,u,DEP_IN])[0]]
            if feasible_seeds:
                seed = max(feasible_seeds, key=lambda u: d0[u])
                r = [DEP_OUT, seed, DEP_IN]
                feas, T_r = compute_schedule(r)
            else:
                print(f"[INFO] Ningún cliente cabe en vehículo cap={cap_lim}. Se descarta ese vehículo.")
                continue

        cap_used = demand[seed]
        unassigned.remove(seed)

        # inserciones secuenciales
        while True:
            cand = []
            for u in list(unassigned):
                ok,pos,C1u,Tn = best_insertion_for(r, T_r, u, cap_used, cap_lim)
                if ok:
                    cand.append((u,pos,C1u,Tn))
            if not cand:
                break
            # C2: balancea lejanía al depósito vs. incremento de C1
            u,pos,C1u,Tn = max(cand, key=lambda x: LAMBDA*d0[x[0]] - x[2])
            r = r[:pos] + [u] + r[pos:]
            T_r = Tn
            cap_used += demand[u]
            unassigned.remove(u)

        routes.append((r, T_r, cap_lim))

    elapsed = time.time()-start
    return routes, elapsed

routes, elapsed = solomon_I1()

# ---------- reporte ----------
total_dist = sum(route_distance(r) for r,_,_ in routes)
# costo operacional: suma de costo variable por distancia + (F + gamma*Q_k) por cada vehículo usado
total_cost = sum(route_operational_cost(r, cap) for r,_,cap in routes)

print("===== SOLUCIÓN SOLOMON I1 (caso base con costos completos) =====")
print(f"Vehículos usados: {len(routes)}")
print(f"Distancia total : {total_dist:,.0f} m")
print(f"Costo total     : {total_cost:,.2f}")
print(f"Tiempo ejec     : {elapsed:.2f} s\n")

for k,(r,T,cap_lim) in enumerate(routes, start=1):
    print(f"Vehículo {k} (cap={cap_lim:.0f}) | Dist: {route_distance(r):,.0f} m | Demanda: {route_demand(r):,.0f}")
    print("Ruta:", " -> ".join(map(str,r)))
    # print("Tiempos:", T)
    print()

# ---------- mapa ----------
m = folium.Map(location=[depot_lat, depot_lon], zoom_start=11)
folium.Marker([depot_lat, depot_lon], tooltip="Depósito", icon=folium.Icon(color='blue')).add_to(m)
palette = ["#"+''.join(random.choice('0123456789ABCDEF') for _ in range(6)) for _ in range(len(routes))]

for idx,(r,_,_) in enumerate(routes):
    pts = [coords[n] for n in r]
    folium.PolyLine(pts, color=palette[idx], weight=3, opacity=0.9).add_to(m)
m.save("routes_map.html")
print("Mapa guardado como routes_map.html")


===== SOLUCIÓN SOLOMON I1 (caso base con costos completos) =====
Vehículos usados: 23
Distancia total : 2,107,075 m
Costo total     : 3,565,542.12
Tiempo ejec     : 11.90 s

Vehículo 1 (cap=10000) | Dist: 52,419 m | Demanda: 9,497
Ruta: 0 -> 96 -> 1 -> 284 -> 53 -> 73 -> 50 -> 188 -> 291 -> 223 -> 293 -> 200 -> 194 -> 305 -> 247 -> 310 -> 54 -> 360 -> 187 -> 60 -> 68 -> 352 -> 236 -> 363 -> 198 -> 103 -> 277 -> 364

Vehículo 2 (cap=10000) | Dist: 81,535 m | Demanda: 9,987
Ruta: 0 -> 348 -> 2 -> 221 -> 147 -> 59 -> 62 -> 109 -> 58 -> 181 -> 283 -> 69 -> 253 -> 95 -> 107 -> 318 -> 298 -> 88 -> 115 -> 364

Vehículo 3 (cap=10000) | Dist: 44,198 m | Demanda: 9,940
Ruta: 0 -> 245 -> 3 -> 244 -> 101 -> 130 -> 110 -> 98 -> 9 -> 264 -> 16 -> 334 -> 18 -> 4 -> 49 -> 315 -> 242 -> 176 -> 309 -> 364

Vehículo 4 (cap=10000) | Dist: 89,370 m | Demanda: 9,915
Ruta: 0 -> 287 -> 342 -> 100 -> 238 -> 326 -> 217 -> 213 -> 336 -> 220 -> 45 -> 204 -> 12 -> 240 -> 243 -> 286 -> 79 -> 94 -> 288 -> 205 -> 246